# SingleR

In [1]:
library_load <- suppressMessages(
    
    suppressWarnings(
    
        list(

            # Seurat 
            library(Seurat), 

            # SingleR
            library(SingleR), 
            library(SingleCellExperiment), 
            
            # Reference
            library(celldex), 

            # Data 
            library(tidyverse), 

            # Pyhton compatibility
            library(reticulate)

        )
    )
)

In [2]:
# Configure reticulate 
use_condaenv(condaenv='p.3.9.19-FD20200109SPLENO', conda="/nobackup/peer/fdeckert/miniconda3/bin/conda", required=NULL)
py_config()

python:         /nobackup/peer/fdeckert/miniconda3/envs/p.3.9.19-FD20200109SPLENO/bin/python
libpython:      /nobackup/peer/fdeckert/miniconda3/envs/p.3.9.19-FD20200109SPLENO/lib/libpython3.9.so
pythonhome:     /nobackup/peer/fdeckert/miniconda3/envs/p.3.9.19-FD20200109SPLENO:/nobackup/peer/fdeckert/miniconda3/envs/p.3.9.19-FD20200109SPLENO
version:        3.9.19 | packaged by conda-forge | (main, Mar 20 2024, 12:50:21)  [GCC 12.3.0]
numpy:          /nobackup/peer/fdeckert/miniconda3/envs/p.3.9.19-FD20200109SPLENO/lib/python3.9/site-packages/numpy
numpy_version:  1.26.4

NOTE: Python version was forced by use_python() function

In [2]:
packageVersion("celldex")

[1] ‘1.14.0’

In [3]:
options(warn=-1)

In [4]:
random_seed <- 42
set.seed(random_seed)

In [5]:
# Set working directory to project root
setwd("/research/peer/fdeckert/FD20200109SPLENO")

In [6]:
# Source files
source("plotting_global.R")

# Parameter settings

In [7]:
# Files 
raw_rds_file <- "data/BSA_0355_SM01_10x_SPLENO/ANALYSIS/raw.rds"

# Plotting Theme
ggplot2::theme_set(theme_global_set(size_select=1)) # From project global source()

# Import Seurat object

In [8]:
so <- readRDS(raw_rds_file)

In [9]:
so <- subset(so, subset=nCount_RNA>=1000)
so <- subset(so, subset=pMt_RNA<=15)

In [38]:
cnt <- GetAssayData(so, assay="RNA", slot="counts")
cnt <- cnt[rowSums(cnt>=3)>=1, ]
so <- CreateSeuratObject(counts=cnt, meta.data=so@meta.data, project="FD20200109SPLENO")

In [39]:
so <- NormalizeData(so, normalization.method="LogNormalize", verbose=FALSE)

# ImmGen 

In [40]:
# Load reference data 
ref <- celldex::ImmGenData()
    
# Seurat object to SingleCellExperiment
sce <- SingleCellExperiment(list(logcounts=GetAssayData(so, assay="RNA", layer="data")))

# Predict labels
label_main_immgen <- SingleR::SingleR(test=sce, ref=ref, labels=ref$label.main, assay.type.test="logcounts", assay.type.ref="logcounts", de.method="classic", fine.tune=TRUE) %>%
    as.data.frame() %>% dplyr::select(pruned.labels, delta.next) %>% dplyr::rename(label_main_immgen=pruned.labels, delta_score_main_immgen=delta.next)

# Haemopedia Mouse RNAseq from Haemosphere

In [ ]:
# Load reference data 
meta <- read.delim("/research/peer/fdeckert/reference/bulkRNAseq/haemosphere/data/Haemopedia-Mouse-RNASeq_samples.txt") %>% tibble::column_to_rownames("sampleId")
cnt <- read.delim("/research/peer/fdeckert/reference/bulkRNAseq/haemosphere/data/Haemopedia-Mouse-RNASeq_raw.txt", row.names=1)[, rownames(meta)]

In [ ]:
# Annotate ENSEML id with gene symbol
gtf <- rtracklayer::import("/research/peer/fdeckert/reference/genome/GRCm38/Mus_musculus.GRCm38.102.gtf") %>% as.data.frame() 

convert <- gtf[, c("gene_id", "gene_name")] %>% dplyr::distinct() %>% dplyr::group_by(gene_name) %>% dplyr::mutate(n=n()) %>% dplyr::filter(n==1) %>% tibble::column_to_rownames("gene_id") %>% dplyr::select(-n)
gene_id <- intersect(rownames(cnt), rownames(convert)) %>% unique()
convert <- convert[which(rownames(convert) %in% gene_id), , drop=FALSE]
cnt <- cnt[which(rownames(cnt) %in% gene_id), , drop=FALSE]
rownames(cnt) <- convert[rownames(cnt), ]

In [ ]:
# Load reference data
ref <- SingleCellExperiment(list(counts=cnt), colData=meta)

# Seurat object to SingleCellExperiment
sce <- SingleCellExperiment(list(counts=GetAssayData(so, assay="RNA", layer="counts")))

# Predict labels
label_main_haemopedia <- SingleR::SingleR(test=sce, ref=ref, labels=ref$celltype, assay.type.test="counts", assay.type.ref="counts", de.method="classic", fine.tune=TRUE) %>%
    as.data.frame() %>% dplyr::select(pruned.labels, delta.next) %>% dplyr::rename(label_main_haemopedia=pruned.labels, delta_score_main_haemopedia=delta.next)

# Save results 

In [ ]:
write.csv(cbind(label_main_immgen, label_main_haemopedia), "result/singler/singler.csv", row.names=TRUE)

# Session info

In [ ]:
sessionInfo()